__<font size="6" weight="bold">Generieren der Testdaten</font>__

Hier werden die Trainingsdaten für den Classificator erstellt. <br>
Als positive Sätze werden für jedes Pattern die empirisch gefundenen Sätze verwendet. <br>
Für die negativen Sätze werden über die Schnittstelle gefundene Anleitungen bei dem das jeweilige Pattern definitiv nicht vorkommt verwendet.
<br>
<br>


## Einbinden der Bibliotheken

In [5]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
import spacy
import re
import pandas as pd
import random
from nltk.corpus import stopwords
#nltk.download('stopwords')

## Verbindung zur Datenbank aufbauen

In [6]:
ap = PlainTextAuthProvider(username='itp_mining', password='mining-data')
cluster = Cluster(['big1.informatik.fh-nuernberg.de'], auth_provider=ap)
session = cluster.connect('itp_mining')

## REST Notebok ausführen

In [7]:
%run REST.ipynb

## Klasse TrainigsSetFactory

__Funktionen:__

__init__(threshold): Initialisierung. Als Parameter kann ein Threshold übergeben werden. Dieser bestimmt aus wie vielen Wörtern ein Satz mindestens bestehen muss. Sätze mit weniger Wörtern werden ignoriert. <br><br>
__trainings_dataframe(pattern_id, isBalanced,has_Stopwords)__: Erstellt ein Trainingsset. <br>Parameter:<br>
pattern_id is vom Datenyp Integer. Bestimmt, zu welchem Pattern ein Dataframe erstellt werden soll.<br> 
is_balanced ist vom Datentyp Boolean. Bestimmt, ob ein Panda Dataframe aus gleich vielen positiven oder negativen Sätzen bestehen soll.   <br>
stopwords_removed ist vom Datentyp Boolean. Bestimmt, ob Stoppwörter entfernt werden sollen.<br>
Return:<br>
Panda Dataframe
<br><br>
__Sentences_to_DF(sentences, isPositive)__: 
<br>Parameter:<br>
sentences: String Liste<br>
isPositive: Boolean<br>
Return:<br> Panda Dataframe<br>
Kovertiert eine String Liste in ein Dataframe um. <br>
Diese Funktion ist eine interne Funktion und soll nicht aufgerufen werden <br><br>
__Get_ALL_PositiveSentences(pattern_id)__: 
<br>Parameter:<br>
pattern_id: Integer<br>
Return:<br> String Liste<br>
liest alle pattern_evidence_sentences aus der Datenbank aus und Tokenisiert diese anschließen mittels spacy.<br>
Diese Funktion ist eine interne Funktion und soll nicht aufgerufen werden <br><br>
__Get_ALL_NegativeSentences(pattern_id)__: 
<br>Parameter:<br>
pattern_id: Integer
Return:<br> String Liste <strong>oder</strong> -1 wenn für das Pattern keine Anleitungen existieren<br>
liest alle Anleitungen aus der Datenbank, bei denen das Pattern sicher nicht vorkommt und Tokenisiert diese anschließen mittels spacy.<br>
Diese Funktion ist eine interne Funktion und soll nicht aufgerufen werden <br><br>
__Convert_evidence_pattern_search_to_game_manualtexts(evidence_json)__:
<br>Parameter:<br>
evidence_json: JSON Liste in der Form:<br>
'evidence_id': 46869, 'evidence_name': 'DIE GUTEN UND DIE BÖSEN GEISTER', 'solr_id': '856d7bfb42220f7e2a8f8b318d07f874'}
Da das erste zurückgegeben JSON in der Datenbank jedoch immer in dieser Form vorliegt:<br>
{'hasallpatterns': None, 'hasnopatterns': None, 'results': 45}<br>
Wird in der Funktion das immer das erste Element gelöscht.<br>
Return:<br>
String List<br>
Diese Funktion ist eine interne Funktion und soll nicht aufgerufen werden <br><br>
__Tokenize(game_manuals_text)__:
<br>Parameter:<br>
game_manuals_text: String Liste.<br>
Return:<br> String Liste<br>
Diese Funktion ist eine interne Funktion und soll nicht aufgerufen werden <br><br>
__remove_stopwords(sentences)__:
<br>Parameter:<br>
sentences: String Liste.<br>
Return:<br> String Liste<br> 
Die Funktion entfernt Stoppwörter, wenn die File "Stopwords.txt" im Ordner "Stopwords" existiert, wird aus diesen Wörtern eine Stopwörter set erzeut, andernfalls wird die Liste von nltk.corpus genutzt.<br>
Diese Funktion ist eine interne Funktion und soll nicht aufgerufen werden <br><br>
__SaveSentencesInFile(dataframe)__:
<br>Parameter:<br>
dataframe: Panda Dataframe<br>
Diese Funktion speichert die positiven und negativen Sätze in ein Textfile "TestSentences.csv" ab.




In [13]:
import pandas as pd
import random
from datetime import datetime


class TrainingSetFactory:
    isValid = 0;
    treshold = int
   
    def __init__(self,treshold=0):
        self.client = empamos_rest_client()
        self.treshold = treshold
        
    def trainings_dataframe(self,pattern_id, isBalanced=True,stopwords_removed=False):
        pos_sentences = self.Get_ALL_PositiveSentences(pattern_id)
        neg_sentences = self.Get_ALL_NegativeSentences(pattern_id)
        
        if neg_sentences == -1:
            self.isValid = 0
            return pd.DataFrame()
        
        if isBalanced:
            if len(pos_sentences) > len(neg_sentences):
                pos_sentences = random.sample(pos_sentences,len(neg_sentences))
            else:
                neg_sentences = random.sample(neg_sentences,len(pos_sentences))
        if stopwords_removed:
            pos_sentences = self.remove_stopwords(pos_sentences)
            neg_sentences = self.remove_stopwords(neg_sentences)
        dfPos = self.Sentences_to_DF(pos_sentences,True)
        dfNeg = self.Sentences_to_DF(neg_sentences,False)
        self.SaveSentencesInFile(dfPos.append(dfNeg))
        self.isValid = 1
        return dfPos.append(dfNeg)
  
    def Sentences_to_DF(self,sentences, isPositive):
        df = pd.DataFrame([sent for sent in sentences])
        df[1] = [int (isPositive) for sent in sentences]
        return df
        
    def Get_ALL_PositiveSentences(self,pattern_id):
        positive_sentences = self.client.get_pattern_evidence_sentences(pattern_id)
        tokenized_sentences = self.Tokenize(positive_sentences)
        return tokenized_sentences
    
    def Get_ALL_NegativeSentences(self,pattern_id):
        games_without_pattern = self.client.evidence_pattern_search('','',pattern_id,'')
        game_text = self.Convert_evidence_pattern_search_to_game_manualtexts(games_without_pattern)  
        if game_text == -1:
            return -1
        tokenized_manuals_sentences = self.Tokenize(game_text)
        return tokenized_manuals_sentences
    
    def Convert_evidence_pattern_search_to_game_manualtexts(self,evidence_json):
        #The first json differs 
        del evidence_json[0]
        if(len(evidence_json) == 0):
            return -1
        manuals = []
        for evidence in evidence_json:
            result = session.execute("select text from working_text where SourceUID = '" + str(evidence["solr_id"]) + "';")
            text = ""
            for line in result:
                text+=line.text
            if len(text.strip()) > 0:
                manuals.append(text) 
        return manuals
    
    def Tokenize(self,game_manuals_text):
        tokenized_sentences = []
        tokenized_treshold_sentences = []
        nlp = spacy.load('de')
        for text in game_manuals_text:
            spacy_doc = nlp(text)
            for sentence in spacy_doc.sents:
                if(self.treshold <= len(sentence)):
                    tokenized_sentences.append(sentence.lemma_)
        return tokenized_sentences
    
    def remove_stopwords(self,sentences):
        sentence_filtert =[]
        if(os.path.isfile("Stopwords/Stopword.txt")):
            STOPWORDS = set(line.strip() for line in open('Stopwords/Stopwords.txt'))
        else:
            STOPWORDS = set(stopwords.words('german'))
        for text in sentences:
            tmp=""
            filtert = [w.lower() for w in text.split(" ") if not w.lower() in STOPWORDS]
            for word in filtert:
                tmp+=word+" "
            sentence_filtert.append(tmp.rstrip())
        return sentence_filtert
        
    def SaveSentencesInFile(self,dataframe):
        f = open("TestSentences.csv","a+")
        dataframe.to_csv(f, header=False)
        f.write("\nSentences were selected on:" + datetime.today().strftime("%d/%m/%Y %H:%M:%S") + "\n\n")
        f.close()